In [ ]:
# Author: Lukas Graner, Niklas Bunzel

import os
from pathlib import Path
from io import BytesIO
import requests
import numpy as np
from skopt import Optimizer
from itertools import product
import PIL.Image
import json

In [ ]:
def load(f):
    return np.asarray(PIL.Image.open(f))/255.0

def scale(img, scale_x, scale_y=None):
    if scale_y is None: scale_y = scale_x
    from scipy.ndimage.interpolation import zoom
    return zoom(img, [scale_x, scale_y, 1], order=0)

def rotate(img, rotation):
    from scipy.ndimage.interpolation import rotate
    return rotate(img, rotation, reshape=False, order=0)

def insert_image(source_img, target_img, target_mask, position_x=0, position_y=0, scale_x=1, scale_y=1, rotation=0, mask_bias=0.5, mask_slope=5):
    scale_x *= source_img.shape[1] / target_img.shape[1]
    scale_y *= source_img.shape[0] / target_img.shape[0]
    position_x *= source_img.shape[1]
    position_y *= source_img.shape[0]
    
    target_img = scale(target_img, scale_x, scale_y)
    target_img = rotate(target_img, rotation)
    target_mask = scale(target_mask, scale_x, scale_y)
    target_mask = rotate(target_mask, rotation)
    
    source_height, source_width = source_img.shape[:2]
    target_height, target_width = target_img.shape[:2]
        
    target_mask = np.tanh((target_mask-mask_bias)*mask_slope)*0.5+0.5
    
    target_height, target_width = target_img.shape[:2]
    offset_y, offset_x = ((np.array(source_img.shape[:2]) - np.array(target_img.shape[:2]))/2 + [position_y, position_x]).astype(np.int32)
    source_slice_y = slice(max(offset_y, 0), min(offset_y+target_height, source_height))
    source_slice_x = slice(max(offset_x, 0), min(offset_x+target_width, source_width))
    target_slice_y = slice(-offset_y if offset_y < 0 else None, source_height-(offset_y+target_height) if (offset_y+target_height > source_height) else None)
    target_slice_x = slice(-offset_x if offset_x < 0 else None, source_width-(offset_x+target_width) if (offset_x+target_width > source_width) else None)
    
    img = np.array(source_img)    
    img[source_slice_y, source_slice_x] = (
        img[source_slice_y, source_slice_x] * (1-target_mask[target_slice_y, target_slice_x]) + 
        target_img[target_slice_y, target_slice_x] * target_mask[target_slice_y, target_slice_x]
    )
    return img

In [ ]:
def predict(img, source, target, api_key="Your-API-Key"):
    with BytesIO() as bstream:
        PIL.Image.fromarray((img*255).astype(np.uint8)).save(bstream, format="png")
        bstream.seek(0)
        request = requests.post(
            f"https://api.mlsec.io/api/facerecognition/submit_sample/?api_token={api_key}&source={source}&target={target}",
            data=bstream.read())
        return request.json()

def insert_and_predict(source, target, position_x=0, position_y=0, scale_x=1, scale_y=1, rotation=0, mask_bias=0.5, mask_slope=5):
    img = insert_image(imgs[source], imgs[target], masks[target], position_x, position_y, scale_x, scale_y, rotation, mask_bias, mask_slope)
    prediction = predict(img, source, target)
    return prediction   

### Load images

In [ ]:
imgs = [load(f) for f in sorted(Path("./faces").glob("*_*.png"))]
masks = [m[...,:3] / m[...,:3].max() for m in map(load, sorted(Path("./facemasks").glob("*_*.png")))]

### Gaussian Optimize

In [ ]:
n_initial_points = 100
n_points = 400

In [ ]:
for source, target in filter(lambda a: a[0]!=a[1], product(range(10), range(10))):
    optimizer = Optimizer(
        [(-0.45, 0.45), (-0.45, 0.45), (0.6, 1.8), (0.6, 1.8), (-30, 30), (0.0, 1), (5, 40)],
        n_initial_points=n_initial_points, initial_point_generator="halton", random_state=0
    )

    steps = []
    for i in range(n_points):        
        next_x = optimizer.ask()
        prediction = insert_and_predict(source, target, *next_x)
        steps.append({"source": source, "target": target, "args": next_x, "prediction": prediction})
        score = prediction["confidence"] + min(0.5, prediction["stealthiness"])
        optimizer_result = optimizer.tell(next_x, -score)
        
    Path(f"results/steps_{source=}_{target=}.jsonl").write_text("\n".join(map(json.dumps, steps)))